In [ ]:
	
from statsbombpy import sb
import pandas as pd
import numpy as np
import networkx as nx
from collections import Counter

In [ ]:
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

In [ ]:
#make a list to hold a df of the events of a match
df_lst = []

for id in events.match_id.unique():
    #find one match for one team
    match_subset = events.loc[events['match_id'] == id]
    match_subset = match_subset.loc[events['possession_team'] == 'Bayer Leverkusen']

    #include only passes and shots
    match_subset = match_subset.loc[(match_subset['type'] == 'Shot') | (match_subset['type'] =='Pass')]

    match_subset['pass_recipient_position'] = None

    match_subset.loc[match_subset['pass_recipient_id'].notna(),'pass_recipient_position'] = match_subset['position'].shift(-1)

    #de næste to linjer kan eventuelt kommenteres ud, og så kan vi bare fjerne na i stedet, hvis vi ikke vil have de ekstra nodes, jeg foreslår :D

    #if a pass does not have a recipient fill the field with the outcome. This allows us to create "out" or "incomplete" nodes if we wish
    match_subset.loc[match_subset['type'] =='Pass', 'pass_recipient_position'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_position']), x['pass_outcome'], x['pass_recipient_position']), axis=1)

    #add shot as a pass recipient as well
    match_subset.loc[match_subset['type'] =='Shot', 'pass_recipient_position'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_position']), x['shot_outcome'], x['pass_recipient_position']), axis=1)


    #sort the values like when we did the passing sequences
    match_subset = match_subset.sort_values(['period','timestamp'], ascending=[True, True])

    df_lst.append(match_subset)   

In [ ]:
#dictionary for the graphs
match_graph_dict = {}

#iterate through the dfs
for match in df_lst:
    match_id = match['match_id'].iloc[0]
    graph = nx.DiGraph()
    edges = []

    #iterate through the passings of the match
    for node in pd.concat([match['position'], match['pass_recipient_position']], axis=0):
        if str(node) not in list(graph.nodes):
            #add the node to the graph
            graph.add_node(str(node))

    for passing in match[['position', 'pass_recipient_position']].itertuples():
        edge = (str(passing.position), str(passing.pass_recipient_position))
        edges.append(edge)

    #count the frequencies
    edges_counter = Counter(edges)

    edge_and_count = [(edge[0], edge[1], edges_counter[edge]) for edge in edges]

    #add weighted edges to graph
    graph.add_weighted_edges_from(edge_and_count)

    #add finished graph to dictionary
    match_graph_dict[match_id] = graph

In [ ]:
nx.draw_networkx(match_graph_dict[3895333])